In [0]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-68-02ff3806b010> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
Exception ignored in:<bound method ExprNode.__del__ of <Expr(h2o.runif <Expr(as.numeric <Expr()#data.hex>)#py_21_sid_b4c3; scalar> -1)#py_24_sid_b4c3; scalar>>
Traceback (most recent call last):
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 187, in __del__
ExprNode.rapids("(rm {})".format(self._cache._id))
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 241, in rapids
return h2o.api("POST /99/Rapids", data={"ast": expr, "session_id": h2o.connection().session_id})
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/h2o.py", line 104, in api
return h2oconn.request(endpoint, data=data, json=json, filename=filename, save_to=save_to)
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/backend/connection.py", line 434, in request
return self._process_response(r

In [3]:
!pip3 install h2o

     |████████████████████████████████| 123.7MB 86kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.11-py2.py3-none-any.whl size=123721708 sha256=0552d4b76906f184d8307360031bee85b4a95c1351f87f9b94190ffe62a25fb4
  Stored in directory: /root/.cache/pip/wheels/71/e6/ed/fb9be80ae2eb0bef9deada74fc370b2532f578d8b00bc1a8fb
Successfully built h2o


In [5]:
import h2o
import numpy as np
import pandas as pd

In [6]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,28 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_dfernandez_lst7yd
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [13]:
import pandas as pd 
df_pandas=pd.read_csv('Input/data.csv')
df=h2o.H2OFrame(df_pandas)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,Ideal,G,SI2,61.9,57,8.44,8.36,5.2,12831
2.43,Very Good,H,SI2,63.2,57,8.56,8.5,5.39,16170
0.8,Premium,F,SI2,61,57,6.03,6.01,3.67,2797
0.4,Ideal,F,I1,63.3,60,4.68,4.64,2.95,630
0.31,Ideal,G,VS2,61.6,55,4.39,4.37,2.7,698
0.53,Very Good,F,SI2,60.5,60,5.19,5.23,3.15,1132
0.7,Premium,D,SI2,60.8,61,5.69,5.64,3.45,1987
1.13,Good,F,SI2,64,58,6.52,6.58,4.19,3952
0.35,Ideal,E,SI1,60.9,55,4.53,4.57,2.77,614
1.01,Fair,H,VS1,61.4,66,6.34,6.24,3.87,5062


In [15]:
df.types

{'carat': 'real',
 'cut': 'enum',
 'color': 'enum',
 'clarity': 'enum',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'int'}

In [16]:
df = df.asnumeric()

In [17]:
df.types

{'carat': 'real',
 'cut': 'real',
 'color': 'real',
 'clarity': 'real',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'real'}

In [18]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,2,3,3,61.9,57,8.44,8.36,5.2,12831
2.43,4,4,3,63.2,57,8.56,8.5,5.39,16170
0.8,3,2,3,61,57,6.03,6.01,3.67,2797
0.4,2,2,0,63.3,60,4.68,4.64,2.95,630
0.31,2,3,5,61.6,55,4.39,4.37,2.7,698
0.53,4,2,3,60.5,60,5.19,5.23,3.15,1132
0.7,3,0,3,60.8,61,5.69,5.64,3.45,1987
1.13,1,2,3,64,58,6.52,6.58,4.19,3952
0.35,2,1,2,60.9,55,4.53,4.57,2.77,614
1.01,0,4,4,61.4,66,6.34,6.24,3.87,5062


In [19]:
y_columns = "price"
x_columns = ["carat","cut","color", "clarity","depth","table","x","y","z"]

In [20]:
train, test = df.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [21]:
test

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,2,3,3,61.9,57,8.44,8.36,5.2,12831
0.4,2,2,0,63.3,60,4.68,4.64,2.95,630
1.13,1,2,3,64,58,6.52,6.58,4.19,3952
0.51,3,1,7,61,61,5.17,5.13,3.14,2060
0.41,2,1,4,62.7,57,4.76,4.72,2.97,1153
1.25,2,1,3,62.3,57,6.83,6.88,4.27,6670
1.01,4,1,6,61.6,58,6.45,6.57,4.01,10693
1,1,3,4,64.4,58,6.23,6.29,4.03,5906
0.26,2,3,4,62.1,55,4.09,4.12,2.55,478
0.71,2,3,3,61.7,55,5.71,5.74,3.53,2266


In [22]:
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

In [23]:
rf_fit1 = rf(model_id='rf_fit1', seed=1)
rf_fit1.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [24]:
rf_fit1.model_performance

Model Details
H2ORandomForestEstimator:Distributed Random Forest
Model Key:rf_fit1


Model Summary:


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 377015.82724253176
RMSE: 614.0161457506894
MAE: 305.5222541737291
RMSLE: 0.10560394636685054
Mean Residual Deviance: 377015.82724253176

Scoring History:

Variable Importances:


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,9454863.0,20.0,20.0,20.0,14371.0,15895.0,15073.2


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-12-09 19:22:38,0.181 sec,0.0,NaN,NaN,NaN
1,,2019-12-09 19:22:39,1.489 sec,1.0,947.128357,453.121652,897052.124662
2,,2019-12-09 19:22:40,1.979 sec,2.0,930.649954,445.375859,866109.337223
3,,2019-12-09 19:22:40,2.274 sec,3.0,870.255682,421.539389,757344.952522
4,,2019-12-09 19:22:41,2.646 sec,4.0,835.442276,405.747669,697963.797222
5,,2019-12-09 19:22:41,2.981 sec,5.0,795.174708,390.999212,632302.816130
6,,2019-12-09 19:22:41,3.289 sec,6.0,793.817299,389.242600,630145.903755
7,,2019-12-09 19:22:42,3.599 sec,7.0,768.235412,379.566658,590185.648341
8,,2019-12-09 19:22:42,3.888 sec,8.0,750.269004,370.861466,562903.579009
9,,2019-12-09 19:22:42,4.154 sec,9.0,738.055105,366.465485,544725.337945


,variable,relative_importance,scaled_importance,percentage
0,y,6.140115e+12,1.000000,0.289409
1,carat,6.022220e+12,0.980799,0.283852
2,x,4.481510e+12,0.729874,0.211232
3,z,2.915536e+12,0.474834,0.137421
4,clarity,8.986752e+11,0.146361,0.042358
5,color,5.202115e+11,0.084723,0.024520
6,depth,9.806961e+10,0.015972,0.004622
7,table,8.815452e+10,0.014357,0.004155
8,cut,5.156992e+10,0.008399,0.002431


<bound method ModelBase.model_performance of >

In [25]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [26]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191209_192303,285105,533.952,285105,273.141,0.0977468
GBM_4_AutoML_20191209_192303,292831,541.138,292831,277.228,0.099009
XGBoost_2_AutoML_20191209_192303,297327,545.277,297327,270.01,0.095797
GBM_1_AutoML_20191209_192303,297370,545.316,297370,289.858,0.110017
GBM_2_AutoML_20191209_192303,299429,547.201,299429,287.99,0.109469
XGBoost_3_AutoML_20191209_192303,301894,549.449,301894,293.957,0.112056
GBM_3_AutoML_20191209_192303,304357,551.686,304357,284.603,0.107161
GBM_5_AutoML_20191209_192303,333895,577.837,333895,289.168,0.100655
XRT_1_AutoML_20191209_192303,372407,610.252,372407,303.991,0.107079
DRF_1_AutoML_20191209_192303,394165,627.826,394165,311.006,0.107693


In [27]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs = 30)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [28]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_1_AutoML_20191209_192303,281198,530.281,281198,277.724,0.103101
GBM_3_AutoML_20191209_192303,281530,530.594,281530,273.951,0.0988578
XGBoost_1_AutoML_20191209_192303,283567,532.51,283567,270.686,0.0916017
GBM_4_AutoML_20191209_192303,284691,533.564,284691,272.033,0.0929295
GBM_2_AutoML_20191209_192303,285029,533.881,285029,278.075,0.10302
XGBoost_2_AutoML_20191209_192303,285053,533.904,285053,266.635,0.0883911
XGBoost_3_AutoML_20191209_192303,289171,537.747,289171,286.61,0.107372
GBM_5_AutoML_20191209_192303,305402,552.632,305402,280.515,nan
XRT_1_AutoML_20191209_192303,342929,585.601,342929,295.96,0.100624
DRF_1_AutoML_20191209_192303,367531,606.244,367531,301.797,0.103119


In [29]:
df2 = pd.read_csv('Input/test.csv')
df2 = h2o.H2OFrame(df2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [30]:
x = df2.drop('id')

In [31]:
x = x.asnumeric()

In [32]:
preds = aml_ti.leader.predict(x)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [33]:
preds2 = preds.as_data_frame(use_pandas=True)

In [34]:
preds2

,predict
0,1393.221264
1,6613.892697
2,1685.012171
3,3828.972968
4,1723.063468
...,...
13480,2129.067823
13481,1975.105278
13482,11417.235820
13483,456.269522


In [35]:
test = df2.as_data_frame(use_pandas=True)

In [36]:
results = pd.DataFrame()
results['id'] = test['id']
results['price'] = preds2['predict']

In [38]:
# Create & upload a file.
results.to_csv('./Output/results.csv',index=False)
